# New Section
Loading a multitask dataset QM8
Models:
1) Multitask with ECPF (better than multitask CM, and DTNN according to Molnet??)
2) Multitask with CM
3) GC
4) DTNN

For each model, look at performing in total, and per task.

Compare to doing a single task separately per task, does that lead to better results?

In [0]:
#load deepchem
%%capture
%tensorflow_version 1.x
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
!conda install -y -c deepchem -c rdkit -c conda-forge -c omnia deepchem-gpu=2.3.0
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [0]:
%cp gaussian_process.py /usr/local/lib/python3.7/site-packages/deepchem/hyper/gaussian_process.py

In [3]:
import os
%env DEEPCHEM_DATA_DIR=/Users/alisa/Dropbox/DeepLearning/DeepChem

env: DEEPCHEM_DATA_DIR=/Users/alisa/Dropbox/DeepLearning/DeepChem


In [4]:
!echo $DEEPCHEM_DATA_DIR

/Users/alisa/Dropbox/DeepLearning/DeepChem


In [6]:
#deepchem modules
import deepchem as dc
from deepchem.models import GraphConvModel, DTNNModel
from deepchem.trans import undo_transforms

#tensorflow and keras modules
import tensorflow as tf
import keras
from tensorflow.keras.layers import Dense, Flatten, Input
from keras.callbacks import ModelCheckpoint


import numpy as np
import pandas as pd


import tempfile
from rdkit import Chem
from rdkit.Chem import Draw
from itertools import islice
from IPython.display import Image, display

%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf







Using TensorFlow backend.


In [7]:
!pip install pyGPGO


     |████████████████████████████████| 2.8 MB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 908 kB 21.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 22.2 MB/s eta 0:00:01
     |████████████████████████████████| 668 kB 25.8 MB/s eta 0:00:01
     |████████████████████████████████| 6.4 MB 11.9 MB/s eta 0:00:01     |█████████▏                      | 1.8 MB 11.9 MB/s eta 0:00:01
     |████████████████████████████████| 294 kB 18.8 MB/s eta 0:00:01
  Created wheel for pyGPGO: filename=pyGPGO-0.4.0.dev1-py3-none-any.whl size=19865 sha256=04d1e7be1728910d22e68b2e5f0cb81c9a0bb50de85a374cb788e099c77e10f7
  Stored in directory: /Users/alisa/Library/Caches/pip/wheels/52/6f/85/71782cff3fc9db678b2ac0d7c3fe44211b3ddeff2aabf4ea57
  Created wheel for theano: filename=Theano-1.0.4-py3-none-any.whl size=2667177 sha256=5d9b3ff83b9947977ceba1b4982f7ca1d7ed4684f58f7e852ec5c5246fb2ff35
  Stored in directory: /Users/alisa/Library/Caches/pip/wheels/33/e0/86/12647586a15bd29c062

In [8]:
import pyGPGO

In [9]:
%pwd

'/Users/alisa/Dropbox (Personal)/DeepLearning/DeepChem'

In [13]:
#load unfeaturized dataset
%mkdir qm8
tasks, datasets, transformers = dc.molnet.load_qm8(featurizer='Raw',save_dir='qm8')

mkdir: cannot create directory ‘qm8’: File exists
Loading dataset from disk.
Loading dataset from disk.
Loading dataset from disk.


In [0]:
!pwd

/content


In [0]:
train, valid, test = datasets

In [12]:
train, valid, test = datasets
train.ids[:5]

NameError: ignored

In [0]:
valid.ids[:5]

array(['[H]C([H])([H])C#N', '[H]C(=O)N([H])[H]',
       '[H]C1=C([H])N([H])C([H])=C1[H]',
       '[H]OC(C([H])([H])[H])(C([H])([H])[H])C([H])([H])[H]',
       '[H]C([H])([H])OC(=O)C([H])([H])[H]'], dtype=object)

In [0]:
test.ids[:5]

array(['[H]C(=O)C([H])=O', '[H]C([H])([H])C([H])([H])C#N',
       '[H]OC1([H])C([H])([H])C1([H])[H]',
       '[H][N][C](OC([H])([H])[H])C([H])([H])[H]',
       '[H]C([H])([H])C#CC([H])([H])C([H])([H])[H]'], dtype=object)

# Multitask

In [10]:
#load dataset from save_dif and apply featurizer to the train, valid and test sets
#test printing train.ids[:5] from raw dataset and featurized one shows they are not the same
#next time, look at test set.
tasks, datasets, transformers = dc.molnet.load_qm8(featurizer='ECFP',save_dir='qm8')


Loading raw samples now.
shard_size: 8192
About to start loading CSV from /Users/alisa/Dropbox/DeepLearning/DeepChem/qm8.csv
Loading shard 1 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 0 took 20.728 s
Loading shard 2 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
Featurizing sample 6000
Featurizing sample 7000
Featurizing sample 8000
TIMING: featurizing shard 1 took 19.890 s
Loading shard 3 of size 8192.
Featurizing sample 0
Featurizing sample 1000
Featurizing sample 2000
Featurizing sample 3000
Featurizing sample 4000
Featurizing sample 5000
TIMING: featurizing shard 2 took 12.948 s
TIMING: dataset construction took 54.335 s
Loading dataset from disk.
TIMING: dataset constr

In [11]:
train, valid, test = datasets
train.ids[:5]



array(['[H]N([H])[H]', '[H]O[H]', '[H]C#C[H]', '[H]C#N', '[H]C([H])=O'],
      dtype=object)

In [0]:
valid.ids[:5]


array(['[H]C1([H])C([H])([H])C1([H])[H]', '[H]C(=O)N([H])C([H])([H])[H]',
       '[H]OC([H])([H])C([H])([H])O[H]',
       '[H]C(=O)C([H])(C([H])([H])[H])C([H])([H])[H]',
       '[H]C([H])([H])C1([H])OC([H])([H])C1([H])[H]'], dtype=object)

In [0]:
test.ids[:5]

array(['[H]C#N', '[H]C1([H])OC1([H])[H]', '[H]C#CC([H])=O',
       '[H]C([H])([H])C([H])([H])C#N',
       '[H]C1([H])C([H])([H])C([H])([H])C1([H])[H]'], dtype=object)

In [0]:
train.X.shape[1]

1024

In [0]:
len(tasks)

16

# Hyperparameter optimization


In [0]:
 
 hps = {
      'layer_sizes': [1500],
      'weight_init_stddevs': [0.02],
      'bias_init_consts': [1.],
      'dropouts': [0.5],
      'penalty': 0.1,
      'batch_size': 50,
      'nb_epoch': 10,
      'learning_rate': 0.001
 }

In [12]:
from deepchem.hyper import GaussianProcessHyperparamOpt 

In [13]:
#regression_metric = dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")
regression_metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
#regression_metric = dc.metrics.Metric(dc.metrics.r2_score)
#len(regression_metric)
#dir(regression_metric)


In [0]:
transformers[0].transform_y


True

In [0]:
y_means, y_stds = train.get_statistics(X_stats=False,y_stats=True)

In [0]:
y_means.shape

(16,)

In [0]:
y_stds.shape

(16,)

In [0]:
original_y = transformers[0].untransform(train.y)

In [0]:
original_y_valid = transformers[0].untransform(valid.y)

In [57]:
original_y.shape

(17428, 16)

In [55]:
train.y.shape

(17428, 16)

In [0]:
import importlib

In [96]:
importlib.reload(dc)

<module 'deepchem' from '/usr/local/lib/python3.7/site-packages/deepchem/__init__.py'>

In [42]:
optimizer = dc.hyper.GaussianProcessHyperparamOpt('tf_regression')
best_hyper_params, best_performance = optimizer.hyperparam_search(
  #  dc.molnet.preset_hyper_parameters.hps['tf_regression'],
    {'layer_sizes': [1000, 1000],
    'weight_init_stddevs': [0.02, 0.02],
     'bias_init_consts': [1., 1.],
     'dropouts': [0.25, 0.25],
     'penalty': 0.0005,
     'penalty_type': 'l2',
     'batch_size': 128,
     'nb_epoch': 10,
     'learning_rate': 0.0008        
    },
    train,
    valid,
    transformers,
    [regression_metric],
    n_tasks = 16,
    direction = False,
    max_iter = 50
)


Evaluation 	 Proposed point 	  Current eval. 	 Best eval.
OrderedDict([('l00', 0.0018844382582277205), ('l01', 366), ('l02', 0.0016057775737480426), ('l03', 1602), ('l04', 2835), ('l05', 0.01606857388581601), ('l06', 0.05514044101693889), ('l07', 2.337430114843099), ('l08', 3.0097541468909377)])
-----------------------------
Start fitting: tf_regression
computed_metrics: [0.031307721030559274, 0.023839943234987334, 0.029267729669591194, 0.06874158139273272, 0.03843314449341854, 0.03084044926988133, 0.05638154914409287, 0.048835701407562326, 0.040975730539198625, 0.030818504792319876, 0.029216197346627944, 0.04069777226886399, 0.03987464241336275, 0.02846475467427706, 0.04983899966779184, 0.08446891645607398]
computed_metrics: [0.03151123524069, 0.02388546590691638, 0.028232854529935587, 0.06819533172552314, 0.03872047517940484, 0.030637068330369523, 0.05658859902264146, 0.048305338641343655, 0.04121398164510888, 0.030600386526453346, 0.028860948970305014, 0.040413557727981636, 0.039979

IndexError: ignored

In [45]:
len(train.w)

17428

In [44]:
model.predict(train)

NameError: ignored

In [39]:
print(best_performance)

-0.30665671788702764


In [40]:
print(best_hyper_params)

{'layer_sizes': [47, 233], 'weight_init_stddevs': [0.06368014710000533, 0.015507302128649411], 'bias_init_consts': [0.9656680951557126, 0.4079071491583006], 'dropouts': [0.25, 0.25], 'penalty': 0.0007165308524767371, 'penalty_type': 'l2', 'batch_size': 135, 'nb_epoch': 5, 'learning_rate': 0.002021174990212994}


In [25]:
#construct a multitask model
%rm -r qm8/fit1
%mkdir qm8/fit1
model_mc_ecfp = dc.models.MultitaskRegressor(
    n_tasks = len(tasks),
    n_features = train.X.shape[1],
    layers = [1770, 402],
    weight_init_stddevs=[0.034862713393011044, 0.06919945210728275],
    bias_init_consts = [1.1775323757179454, 1.7718493844825818],
    dropouts = [0.25, 0.25],
    penalty = 0.0015523311536663354,
    penalty_type = 'l2',
    batch_size = 157,
    nb_epoch = 10,
    learning_rate = 0.0030848283520802013,
    model_dir = 'qm8/fit1'
)

In [0]:
#callback function for checking validation loss on_epoch_end



 # def on_batch_end(self, batch, logs=None):
def on_batch_end(self, current_step):
    regression_metric = [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")]

    current_batch = np.ceil(current_step / self.batch_size)
    if (current_step % self.batch_size == 0):
       print("Evaluating model, batch #=",current_batch)
       train_scores = model_mc_ecfp.evaluate(train, regression_metric, transformers)
       train_scores['mean_absolute_error']
       train_scores_values = np.array(train_scores['mean_absolute_error'])
       print('Mean training MAE over all tasks: {:7.4f}'.format(np.sum(train_scores_values)))
       valid_scores = model_mc_ecfp.evaluate(valid, regression_metric, transformers)
       valid_scores_values = np.array(valid_scores['mean_absolute_error'])
       print('Mean validation MAE over all tasks: {:7.4f}'.format(np.sum(valid_scores_values)))


#checkpoint_name = 'Weights-{epoch:03d}--{valid_scores:.5f}.hdf5' 
#checkpoint = ModelCheckpoint(checkpoint_name, monitor='valid_scores', verbose = 1, save_best_only = True, mode ='auto')
#callbacks_list = [checkpoint, LossAndErrorPrintingCallback]
callbacks_list = [on_batch_end]


In [34]:
len(train.w)

17428

In [27]:
model_mc_ecfp.fit(train)

0.229865534522105

In [0]:
#find and load best checking point, starting from 9 and going back
def find_best_checkpoint(model,model_dir,dataset,transformers):
  best_valid = float("inf") 
  metric = dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")
  for checkpoint_name in model.get_checkpoints(model_dir=model_dir):
    model.restore(checkpoint = checkpoint_name, model_dir = model_dir, session= None)
    valid_scores = model.evaluate(dataset, [metric], transformers)
    valid_scores_mean = np.sum(np.array(valid_scores['mean_absolute_error']))
    if valid_scores_mean < best_valid:
      best_checkpoint = checkpoint_name
    best_valid = min(best_valid,valid_scores_mean)
  print('found best checkpoint:',best_checkpoint,'with MAE sum=',np.sum(best_valid)) 
  model.restore(checkpoint = best_checkpoint, model_dir = model_dir, session= None)
  return 




  
  


In [42]:
y_pred = model_mc_ecfp.predict(train, transformers)

In [43]:
len(y_pred)

17443

In [47]:
transformers[0].__class__


deepchem.trans.transformers.NormalizationTransformer

In [39]:
len(train.w)

17428

In [40]:
y_undo = dc.deepchem.trans.undo_transforms(train.y, transformers)

In [41]:
len(y_undo)

17428

In [35]:
train_mae = model_mc_ecfp.evaluate(train, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(train_mae['mean_absolute_error'])


IndexError: boolean index did not match indexed array along dimension 0; dimension is 17443 but corresponding boolean dimension is 17428

In [34]:
valid_mae = model_mc_ecfp.evaluate(valid, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(valid_mae['mean_absolute_error'])

computed_metrics: [0.011155001887996587, 0.011671415661965927, 0.017807233539442614, 0.03723932708795348, 0.011130501433203079, 0.011636315171720778, 0.016030276810443878, 0.030063601422979796, 0.011094268480505994, 0.011537161529457845, 0.01603570988490113, 0.03007907342823597, 0.010795664205263875, 0.010856304078821074, 0.018541971514212566, 0.03266323616256072]


0.28833706229966527

In [35]:
test_mae = model_mc_ecfp.evaluate(test, [dc.metrics.Metric(dc.metrics.mean_absolute_error, mode="regression")], transformers)
np.sum(test_mae['mean_absolute_error'])

computed_metrics: [0.010957036263989318, 0.01210697745125941, 0.01984345470170992, 0.037290666325827795, 0.011159215840090877, 0.011732288000184153, 0.01719291419262194, 0.03059921490062259, 0.011157291428647723, 0.011637534857502919, 0.01719620835340863, 0.03061176995745235, 0.010511097908340834, 0.011011253599299685, 0.01972109686168274, 0.03307469803127647]


0.2958027186739174

In [0]:

def print_nice_table(tasks,test_mae):
  print('  task  ','   MAE   ')
  for task in tasks:
      print('  {0}  {1:7.4f}   '.format(task,test_mae['mean_absolute_error'][tasks.index(task)]))
  return

In [0]:
print(test_maetasks.index('E1-CC2'))

0


In [37]:
print_nice_table(tasks,test_mae)

  task      MAE   
  E1-CC2   0.0110   
  E2-CC2   0.0121   
  f1-CC2   0.0198   
  f2-CC2   0.0373   
  E1-PBE0   0.0112   
  E2-PBE0   0.0117   
  f1-PBE0   0.0172   
  f2-PBE0   0.0306   
  E1-PBE0   0.0112   
  E2-PBE0   0.0117   
  f1-PBE0   0.0172   
  f2-PBE0   0.0306   
  E1-CAM   0.0105   
  E2-CAM   0.0110   
  f1-CAM   0.0197   
  f2-CAM   0.0331   


In [95]:
score = [5, 2]
if isinstance(score, list):
  score = np.sum(np.array(score))
print(score)

7
